In [ ]:
# Parameters for multi-orbital tight-binding model of armchair CNTs
# Physical parameters
r0 = 1.42          # Equilibrium carbon-carbon bond length (Å)
max_dist = 2.5     # Maximum distance for nearest-neighbor interactions (Å)
r_ref = np.sqrt(3) * r0  # Reference distance for lattice calculations (Å)

# Hopping parameters (eV), based on Reich et al. (2004)
Vpp_sigma = 6.4   # p-p sigma hopping
Vpp_pi = -2.7     # p-p pi hopping
Vsp_sigma = 6.0   # s-p sigma hopping
Vss_sigma = -6.8  # s-s sigma hopping
beta = 4.46       # Decay parameter for hopping integrals

# Overlap parameters (dimensionless), based on Popov and Henrard (2004)
Vss_sigma_overlap = -0.2  # s-s sigma overlap
Vsp_sigma_overlap = 0.1   # s-p sigma overlap
Vpp_sigma_overlap = 0.2   # p-p sigma overlap
Vpp_pi_overlap = -0.05    # p-p pi overlap
beta_overlap = 2.85       # Decay parameter for overlap integrals

# Simulation settings
Nk = 500          # Number of k-points for band structure
NUM_BINS = 300    # Number of energy bins for DOS

# Plot settings for publication-quality figures
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['DejaVu Serif', 'Times New Roman', 'serif'],
    'mathtext.fontset': 'cm',  # Computer Modern math fonts
    'font.size': 12,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
})


In [ ]:
def compute_Ch(n, m):
    """
    Compute the chiral vector, translation vector, and number of atoms in the unit cell
    for an armchair carbon nanotube with chirality (n,m).

    Args:
        n (int): Chiral index n.
        m (int): Chiral index m.

    Returns:
        tuple: (Ch_vec, T_vec, N_atom_cell, a1, a2)
            - Ch_vec: Chiral vector (2D numpy array).
            - T_vec: Translation vector (2D numpy array).
            - N_atom_cell: Number of carbon atoms in the unit cell.
            - a1, a2: Graphene lattice vectors (2D numpy arrays).
    """
    sqrt3 = np.sqrt(3)
    a1 = r0 * np.array([sqrt3/2, 1/2])  # First graphene lattice vector
    a2 = r0 * np.array([sqrt3/2, -1/2])  # Second graphene lattice vector
    Ch_vec = n * a1 + m * a2  # Chiral vector C_h = n*a1 + m*a2

    # Compute translation vector T using greatest common divisor (gcd)
    d = gcd(2*n + m, 2*m + n)
    T1 = (2*m + n) // d
    T2 = -(2*n + m) // d
    T_vec = T1 * a1 + T2 * a2  # Translation vector T = T1*a1 + T2*a2

    # Number of atoms in the unit cell: 4 * (n^2 + m^2 + n*m) / d
    N_atom_cell = 4 * (n**2 + m**2 + n*m) // d
    return Ch_vec, T_vec, N_atom_cell, a1, a2

def generate_CNT_lattice(n, m, include_B=True, eps=1e-6):
    """
    Generate 2D lattice coordinates for an armchair CNT unit cell by rolling a graphene sheet.

    Args:
        n (int): Chiral index n.
        m (int): Chiral index m.
        include_B (bool): Include B-sublattice atoms (default: True).
        eps (float): Numerical tolerance for boundary conditions (default: 1e-6).

    Returns:
        tuple: (unique_positions, None, Ch_vec, T_vec)
            - unique_positions: Unique 2D coordinates of atoms in the unit cell.
            - None: Placeholder for sublattice info (not used).
            - Ch_vec: Chiral vector.
            - T_vec: Translation vector.
    """
    # Compute chiral and translation vectors
    Ch_vec, T_vec, N_theoretical, a1, a2 = compute_Ch(n, m)
    M = np.column_stack((Ch_vec, T_vec))  # Matrix of basis vectors
    invM = np.linalg.inv(M)  # Inverse for fractional coordinates

    # Define search range for lattice points
    L = int(np.ceil(np.linalg.norm(Ch_vec)/r0)) + 10
    positions = []

    for t1 in range(-L, L+1):
        for t2 in range(-L, L+1):
            # A-sublattice position
            pos_A = t1 * a1 + t2 * a2
            frac = invM.dot(pos_A)
            if (0 <= frac[0] < 1) and (0 <= frac[1] < 1):
                frac_mod = frac - np.floor(frac)  # Fold into unit cell
                frac_mod = np.where(frac_mod >= 1 - eps, 0, frac_mod)
                pos_folded = np.round(M.dot(frac_mod), decimals=6)
                positions.append(pos_folded)
            
            # B-sublattice position (if included)
            if include_B:
                pos_B = pos_A + (a1 + a2) / 3
                fracB = invM.dot(pos_B)
                if (0 <= fracB[0] < 1) and (0 <= fracB[1] < 1):
                    fracB_mod = fracB - np.floor(fracB)
                    fracB_mod = np.where(fracB_mod >= 1 - eps, 0, fracB_mod)
                    pos_B_folded = np.round(M.dot(fracB_mod), decimals=6)
                    positions.append(pos_B_folded)
    
    positions = np.array(positions)
    unique_positions = np.unique(positions, axis=0)  # Remove duplicates
    
    # Verify number of atoms matches theoretical value
    assert len(unique_positions) == N_theoretical, "Number of atoms does not match theoretical value"
    print(f'unique positions = {len(unique_positions)}')
    print(f'theoretical number of atom = {N_theoretical}')
    return unique_positions, None, Ch_vec, T_vec

def get_3d_pos(th, z, R):
    """
    Convert cylindrical coordinates to 3D Cartesian coordinates for a CNT.

    Args:
        th (float): Angular coordinate (radians).
        z (float): Axial coordinate (Å).
        R (float): CNT radius (Å).

    Returns:
        tuple: (x, y, z) Cartesian coordinates (Å).
    """
    x = R * np.cos(th)
    y = R * np.sin(th)
    return x, y, z

def get_3d_vec(i, j, tharr, zarr, R, shift_z=0.0):
    """
    Compute the 3D vector between two atoms in a CNT.

    Args:
        i (int): Index of first atom.
        j (int): Index of second atom.
        tharr (array): Array of angular coordinates.
        zarr (array): Array of axial coordinates.
        R (float): CNT radius (Å).
        shift_z (float): Axial shift for inter-unit-cell interactions (default: 0.0).

    Returns:
        tuple: (rx, ry, rz) 3D vector components (Å).
    """
    x1, y1, z1 = get_3d_pos(tharr[i], zarr[i], R)
    x2, y2, z2 = get_3d_pos(tharr[j], zarr[j] + shift_z, R)
    return (x2 - x1, y2 - y1, z2 - z1)

def cylindrical_direction(rx, ry, rz):
    """
    Compute direction cosines and distance for a 3D vector in cylindrical coordinates.

    Args:
        rx, ry, rz (float): Cartesian components of the vector (Å).

    Returns:
        tuple: (l, m, n, dist)
            - l, m: Direction cosines in the cylindrical plane.
            - n: Direction cosine along the axial direction.
            - dist: Vector magnitude (Å).
    """
    r_cyl = np.hypot(rx, ry)  # Cylindrical radius
    if r_cyl < 1e-8:
        return 0, 0, 0, 0  # Handle zero cylindrical radius
    l = rx / r_cyl
    m = ry / r_cyl
    dist = np.sqrt(rx*rx + ry*ry + rz*rz)
    if abs(dist) < 1e-12:
        return 0, 0, 0, dist  # Handle zero distance
    n = rz / dist
    return l, m, n, dist

def sk_matrix(rx, ry, rz, is_first_nn=True):
    """
    Construct Hamiltonian and overlap matrix blocks using the Slater-Koster method.

    Args:
        rx, ry, rz (float): Cartesian components of the interatomic vector (Å).
        is_first_nn (bool): Flag for first nearest-neighbor interactions (default: True).

    Returns:
        tuple: (H_block, S_block)
            - H_block: 4x4 Hamiltonian block for s, p_x, p_y, p_z orbitals.
            - S_block: 4x4 Overlap matrix block.
    """
    l, m, n, dist = cylindrical_direction(rx, ry, rz)
    if dist < 1e-12:
        return np.zeros((4,4), dtype=complex), np.eye(4)  # Zero distance case

    # Compute distance-dependent hopping integrals
    vsig = Vpp_sigma * np.exp(-beta * ((dist / r0) - 1))
    vpi  = Vpp_pi    * np.exp(-beta * ((dist / r0) - 1))
    vsp  = Vsp_sigma * np.exp(-beta * ((dist / r0) - 1))
    vss  = Vss_sigma * np.exp(-beta * ((dist / r0) - 1))

    # Compute distance-dependent overlap integrals
    s_vsig = Vpp_sigma_overlap * np.exp(-beta_overlap * ((dist / r0) - 1))
    s_vpi  = Vpp_pi_overlap    * np.exp(-beta_overlap * ((dist / r0) - 1))
    s_vsp  = Vsp_sigma_overlap * np.exp(-beta_overlap * ((dist / r0) - 1))
    s_vss  = Vss_sigma_overlap * np.exp(-beta_overlap * ((dist / r0) - 1))

    H_block = np.zeros((4,4), dtype=complex)
    S_block = np.zeros((4,4), dtype=complex)

    # Hamiltonian entries (s, p_x, p_y, p_z orbitals)
    H_block[0,0] = vss
    H_block[0,1] = l * vsp
    H_block[0,2] = m * vsp
    H_block[0,3] = n * vsp
    H_block[1,1] = (l*l)*vsig + (1 - l*l)*vpi
    H_block[2,2] = (m*m)*vsig + (1 - m*m)*vpi
    H_block[3,3] = (n*n)*vsig + (1 - n*n)*vpi
    H_block[1,2] = H_block[2,1] = l*m*(vsig - vpi)
    H_block[1,3] = H_block[3,1] = l*n*(vsig - vpi)
    H_block[2,3] = H_block[3,2] = m*n*(vsig - vpi)

    # Overlap matrix entries
    S_block[0,0] = s_vss
    S_block[0,1] = l * s_vsp
    S_block[0,2] = m * s_vsp
    S_block[0,3] = n * s_vsp
    S_block[1,1] = (l*l)*s_vsig + (1 - l*l)*s_vpi
    S_block[2,2] = (m*m)*s_vsig + (1 - m*m)*s_vpi
    S_block[3,3] = (n*n)*s_vsig + (1 - n*n)*s_vpi
    S_block[1,2] = S_block[2,1] = l*m*(s_vsig - s_vpi)
    S_block[1,3] = S_block[3,1] = l*n*(s_vsig - s_vpi)
    S_block[2,3] = S_block[3,2] = m*n*(s_vsig - s_vpi)

    return H_block, S_block

def build_hamiltonian(n, m):
    """
    Build the k-dependent Hamiltonian and overlap matrices for a CNT.

    Args:
        n (int): Chiral index n.
        m (int): Chiral index m.

    Returns:
        tuple: (H_of_k, info)
            - H_of_k: Function to compute Hamiltonian and overlap matrices at wavevector k.
            - info: Dictionary with CNT parameters (theta_list, z_list, R, T_length, N_cell, Ch_vec).
    """
    coords_2D, subl, Ch_vec, T_vec = generate_CNT_lattice(n, m, True)
    N_cell = len(coords_2D)

    # Compute CNT radius and unit cell length
    R = (2.46/(2*np.pi)) * math.sqrt(n*n + n*m + m*m)
    T_length = np.linalg.norm(T_vec)

    # Define unit vectors along chiral and translation directions
    hatC = Ch_vec / np.linalg.norm(Ch_vec)
    hatT = T_vec / np.linalg.norm(T_vec)

    # Compute cylindrical coordinates (theta, z) for each atom
    tharr = []
    zarr  = []
    for rr in coords_2D:
        th = (2*math.pi / np.linalg.norm(Ch_vec)) * (rr.dot(hatC))
        zz = rr.dot(hatT)
        tharr.append(th)
        zarr.append(zz)

    tharr = np.array(tharr)
    zarr  = np.array(zarr)

    # Collect first nearest neighbors within the same unit cell
    nb_in = []
    for i in range(N_cell):
        for j in range(i+1, N_cell):
            rx, ry, rz = get_3d_vec(i, j, tharr, zarr, R)
            dd = np.sqrt(rx**2 + ry**2 + rz**2)
            if dd < max_dist:
                nb_in.append((i, j))

    # Collect first nearest neighbors across unit cells
    nb_int = []
    for i in range(N_cell):
        for j in range(N_cell):
            rx, ry, rz = get_3d_vec(i, j, tharr, zarr, R, shift_z=T_length)
            dd = np.sqrt(rx**2 + ry**2 + rz**2)
            if dd < max_dist:
                nb_int.append((i, j, T_length))

    def place_block(H, S, i4, j4, subH, subS):
        """Helper function to place 4x4 Hamiltonian and overlap blocks."""
        H[i4:i4+4, j4:j4+4] += subH
        H[j4:j4+4, i4:i4+4] += subH.T
        S[i4:i4+4, j4:j4+4] += subS
        S[j4:j4+4, i4:i4+4] += subS.T

    def H_of_k(kpar):
        """Compute Hamiltonian and overlap matrices at wavevector kpar."""
        Hdim = 4 * N_cell
        Sdim = 4 * N_cell
        H = np.zeros((Hdim, Hdim), dtype=complex)
        S = np.zeros((Sdim, Sdim), dtype=complex)

        # Intra-unit-cell first nearest neighbors
        for (i, j) in nb_in:
            rx, ry, rz = get_3d_vec(i, j, tharr, zarr, R, shift_z=0.0)
            Hblock, Sblock = sk_matrix(rx, ry, rz, is_first_nn=True)
            i4, j4 = 4*i, 4*j
            place_block(H, S, i4, j4, Hblock, Sblock)

        # Inter-unit-cell first nearest neighbors
        for (i, j, dz) in nb_int:
            rx, ry, rz = get_3d_vec(i, j, tharr, zarr, R, shift_z=dz)
            Hblock, Sblock = sk_matrix(rx, ry, rz, is_first_nn=True)
            phase = np.exp(1j * kpar * dz)  # Bloch phase factor
            i4, j4 = 4*i, 4*j
            H[i4:i4+4, j4:j4+4] += Hblock * phase
            H[j4:j4+4, i4:i4+4] += Hblock.conj().T * phase.conj()
            S[i4:i4+4, j4:j4+4] += Sblock * phase
            S[j4:j4+4, i4:i4+4] += Sblock.conj().T * phase.conj()
        return H, S

    info = {
        'theta_list': tharr,
        'z_list': zarr,
        'R': R,
        'T_length': T_length,
        'N_cell': N_cell,
        'Ch_vec': Ch_vec
    }
    return H_of_k, info

def solve_with_lowdin(H, S):
    eigvals_s, U = np.linalg.eigh(S)
    min_eig_s = np.min(eigvals_s)

    if min_eig_s < 1e-5:
        regularization = max(1e-5 - min_eig_s, 0.0)
        S_reg = S + regularization * np.eye(S.shape[0])
        eigvals_s, U = np.linalg.eigh(S_reg)

    Lambda_inv_sqrt = np.diag(1.0 / np.sqrt(eigvals_s))
    S_inv_sqrt = U @ Lambda_inv_sqrt @ U.conj().T
    H_ortho = S_inv_sqrt @ H @ S_inv_sqrt
    eigvals_h, eigvecs_h = np.linalg.eigh(H_ortho)
    return eigvals_h, eigvecs_h

def compute_band_structure(builder, TL, Nk):
    k_vals = np.linspace(-np.pi / TL, np.pi / TL, Nk)
    Hk0, Sk0 = builder(0)
    dim = Hk0.shape[0]
    Ek = np.zeros((Nk, dim), dtype=float)

    conduction_band_min = float('inf')
    valence_band_max = float('-inf')

    for i, k in enumerate(k_vals):
        Hk, Sk = builder(k)

        eigvals_s = np.linalg.eigvalsh(Sk)
        min_S = np.min(eigvals_s)
        if min_S < 1e-5:
            offset = 1e-5 - min_S
            Sk = Sk + np.eye(dim) * offset

        eigvals, _ = solve_with_lowdin(Hk, Sk)
        Ek[i, :] = eigvals
        cond_vals = eigvals[eigvals > 0]
        val_vals  = eigvals[eigvals < 0]
        if len(cond_vals) > 0:
            c_local_min = np.min(cond_vals)
            if c_local_min < conduction_band_min:
                conduction_band_min = c_local_min
        if len(val_vals) > 0:
            v_local_max = np.max(val_vals)
            if v_local_max > valence_band_max:
                valence_band_max = v_local_max

    if conduction_band_min == float('inf') or valence_band_max == float('-inf'):
        gap = 0.0
    else:
        gap = conduction_band_min - valence_band_max

    return k_vals, Ek, gap

def compute_DOS(Hbuilder, TL, Nk, Nb=300):
    k_vals = np.linspace(-np.pi / TL, np.pi / TL, Nk)
    all_eigvals = []
    for k_par in k_vals:
        Hk, Sk = Hbuilder(k_par)
        eigvals_s = np.linalg.eigvalsh(Sk)
        min_S = np.min(eigvals_s)
        if min_S < 1e-2:
            offset = 1e-2 - min_S
            Sk += np.eye(Sk.shape[0]) * offset

        eigvals, _ = solve_with_lowdin(Hk, Sk)
        all_eigvals.extend(eigvals)

    all_eigvals = np.array(all_eigvals)
    E_min, E_max = -15, 15
    energy_grid = np.linspace(E_min, E_max, Nb)
    dos = np.zeros_like(energy_grid)
    BROADENING = 0.05
    for E in all_eigvals:
        gf = np.exp(-0.5 * ((energy_grid - E) / BROADENING) ** 2)
        gf /= (BROADENING * np.sqrt(2 * np.pi))
        dos += gf
    return energy_grid, dos


def compute_DOS_with_projection(Hbuilder, TL, Nk, Nb=300):
    k_vals = np.linspace(-np.pi / TL, np.pi / TL, Nk)
    builder, info = build_hamiltonian(n, m)
    N_cell = info['N_cell']
    eigenvalues = []
    orbital_weights = []
    for k_par in k_vals:
        Hk, Sk = Hbuilder(k_par)
        eigvals_s = np.linalg.eigvalsh(Sk)
        min_S = np.min(eigvals_s)
        if min_S < 1e-5:
            offset = 1e-5 - min_S
            Sk = Sk + np.eye(Sk.shape[0]) * offset
        eigvals, eigvecs = solve_with_lowdin(Hk, Sk)
        for iband, E in enumerate(eigvals):
            psi = eigvecs[:, iband]
            dim = psi.shape[0]
            w_s = 0.0
            w_px = 0.0
            w_py = 0.0
            w_pz = 0.0
            for iatom in range(N_cell):
                base = iatom * 4
                c_s  = psi[base + 0]
                c_px = psi[base + 1]
                c_py = psi[base + 2]
                c_pz = psi[base + 3]
                w_s  += np.abs(c_s)**2
                w_px += np.abs(c_px)**2
                w_py += np.abs(c_py)**2
                w_pz += np.abs(c_pz)**2
            eigenvalues.append(E)
            orbital_weights.append([w_s, w_px, w_py, w_pz])
    eigenvalues = np.array(eigenvalues)
    orbital_weights = np.array(orbital_weights)
    E_min, E_max = -13, 13
    energy_grid = np.linspace(E_min, E_max, Nb)
    BROADENING = 0.05
    s_dos  = np.zeros_like(energy_grid)
    px_dos = np.zeros_like(energy_grid)
    py_dos = np.zeros_like(energy_grid)
    pz_dos = np.zeros_like(energy_grid)
    for iband, E in enumerate(eigenvalues):
        w_s, w_px, w_py, w_pz = orbital_weights[iband]
        gf = np.exp(-0.5 * ((energy_grid - E)/BROADENING)**2)
        gf /= (BROADENING * np.sqrt(2.0 * np.pi))
        s_dos  += w_s  * gf
        px_dos += w_px * gf
        py_dos += w_py * gf
        pz_dos += w_pz * gf
    s_dos /= Nk
    px_dos /= Nk
    py_dos /= Nk
    pz_dos /= Nk
    total_dos = s_dos + px_dos + py_dos + pz_dos
    return energy_grid, s_dos, px_dos, py_dos, pz_dos, total_dos

def save_plot(directory, filename):
    base_path = os.path.join(os.getcwd(), "results")
    full_directory = os.path.join(base_path, directory)
    os.makedirs(full_directory, exist_ok=True)
    count = 1
    filepath = os.path.join(full_directory, f"{filename}_{count}.png")
    while os.path.exists(filepath):
        count += 1
        filepath = os.path.join(full_directory, f"{filename}_{count}.png")
    plt.savefig(filepath)
    return filepath

def plot_band_structure(kv, Ek, n, m, dia, Nc, TL):
    plt.figure(figsize=(10, 5))
    num_bands = Ek.shape[1]
    max_band_plot = num_bands  
    for i in range(max_band_plot):
        plt.plot(kv, Ek[:, i], 'b-', linewidth=0.8)

    conduction_band_min = np.min(Ek[Ek > 0]) if np.any(Ek > 0) else None
    valence_band_max    = np.max(Ek[Ek < 0]) if np.any(Ek < 0) else None
    if conduction_band_min is not None and valence_band_max is not None:
        gap = conduction_band_min - valence_band_max
    else:
        gap = 0.0
    plt.ylabel("Energy (eV)", fontsize=25)
    plt.axhline(0, color='k', ls='--', lw=0.5)
    plt.grid(True)

    # -pi/T ~ pi/T
    X_val = np.pi / TL
    plt.xticks([-X_val, 0, X_val], ["-X", r"$\Gamma$", "X"], fontsize=25)
    plt.yticks(fontsize=25)
    plt.ylim(-10, 10)

    filepath = save_plot("band_structures", f"band_structure_{n}_{m}")
    plt.tight_layout()
    print(f"Computed Band Gap (global) : {gap:.6f} eV")
    return filepath

def plot_pdos(energy_grid, s_dos, px_dos, py_dos, pz_dos, total_dos, n, m, dia):
    plt.figure(figsize=(10, 5))
    plt.plot(energy_grid, total_dos, label='Total DOS', color='k', linewidth=2)
    plt.plot(energy_grid, s_dos,      label='s orbital', color='r')
    plt.plot(energy_grid, px_dos,     label='p_x orbital', color='g')
    plt.plot(energy_grid, py_dos,     label='p_y orbital', color='b')
    plt.plot(energy_grid, pz_dos,     label='p_z orbital', color='m')
    plt.xlabel("Energy (eV)", fontsize=25)
    plt.ylabel(r"$\rho$ (E)", fontsize=25)
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.axhline(0, color='gray', ls='--', lw=0.8)
    plt.legend(fontsize=15)
    plt.grid(True)

    filepath = save_plot("pdos", f"pdos_{n}_{m}")
    return filepath

def simulate_CNT(n, m, plot_type=None, show_plot=True):
    builder, info = build_hamiltonian(n, m)
    Nc = info['N_cell']
    TL = info['T_length']
    Ch_vec = info['Ch_vec']
    R = info['R']
    dia = 2*R

    if plot_type == 'band':
        kv, Ek, gap = compute_band_structure(builder, TL, Nk)
        if show_plot:
            filepath = plot_band_structure(kv, Ek, n, m, dia, Nc, TL)
        else:
            filepath = None
        return {'k_vals': kv, 'E_k': Ek, 'gap_min': gap, 'saved_path': filepath}
    
    elif plot_type == 'pdos':
        eng, s_dos, px_dos, py_dos, pz_dos, total_dos = compute_DOS_with_projection(builder, TL, Nk, Nb=300)
        if show_plot:
            filepath = plot_pdos(eng, s_dos, px_dos, py_dos, pz_dos, total_dos, n, m, dia)
        else:
            filepath = None
        return {'energy grid': eng, 's dos': s_dos, 'px_dos': px_dos,'py_dos':py_dos,'pz_dos':pz_dos, 'total dos':total_dos,'saved_path': filepath}

    else:
        print(f"(n,m)=({n},{m}), diameter={dia:.2f} Å")
        return {'n': n, 'm': m, 'dia': dia, 'N_cell': Nc}
    
if __name__ == "__main__":
    n, m = 4, 4
    result = simulate_CNT(n, m, plot_type='pdos', show_plot=True)